***내부 인덱스란?***
> **내부 인덱스**는 OR-Tools 내부에서 사용하는 자체적인 노드 번호 체계입니다.  
> 사용자가 지정한 노드 번호(예: 0, 1, 2, ...)와 다를 수 있습니다.

---
예를 들어 차량이 2대이고 고객이 4명이라면, 내부적으로는 아래와 같은 번호 체계를 가질 수 있습니다:

| 내부 인덱스 | 의미                | 사용자 노드 번호 |
|-------------|---------------------|------------------|
| 0           | 차량 0의 시작점     | 0 (depot)        |
| 1           | 차량 1의 시작점     | 0 (depot)        |
| 2           | 고객 1              | 1                |
| 3           | 고객 2              | 2                |
| ...         | ...                 | ...              |

---

인덱스 변환 함수   

| 함수                         | 설명                                             |
|------------------------------|--------------------------------------------------|
| `manager.IndexToNode(i)`     | 내부 인덱스 `i` → 사용자 노드 번호로 변환         |
| `manager.NodeToIndex(j)`     | 사용자 노드 번호 `j` → 내부 인덱스로 변환         |


### 1. Vehicle Routing Problem (VRP)
차량 개수 

In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Stores the data for the problem."""
    #data 딕셔너리 생성 
    data = {}

    #data['distance_matrix][i][j]: i에서 j까지 거리
    data["distance_matrix"] = [
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    ]

    data["num_vehicles"] = 1 #차량이 한대 
    data["depot"] = 0
    return data
data=create_data_model()
#색인 관리자, 일종의 정보 관리자
#솔버 내부 인덱스를 사용자 위치 번호로 변환하는 등 인덱스를 관리함
manager= pywrapcp.RoutingIndexManager(
    len(data['distance_matrix']),#전체 노드 수(고객 수+출발지)
    data['num_vehicles'], #사용할 차량 수
    data['depot'] #출발지 노드 인덱스
)


#Routing : 경로 최적화를 위한 핵심 솔버 객체 
routing= pywrapcp.RoutingModel(manager)

load C:\Users\paint\AppData\Roaming\Python\Python39\site-packages\ortools\.libs\zlib1.dll...
load C:\Users\paint\AppData\Roaming\Python\Python39\site-packages\ortools\.libs\abseil_dll.dll...
load C:\Users\paint\AppData\Roaming\Python\Python39\site-packages\ortools\.libs\utf8_validity.dll...
load C:\Users\paint\AppData\Roaming\Python\Python39\site-packages\ortools\.libs\re2.dll...
load C:\Users\paint\AppData\Roaming\Python\Python39\site-packages\ortools\.libs\libprotobuf.dll...
load C:\Users\paint\AppData\Roaming\Python\Python39\site-packages\ortools\.libs\highs.dll...
load C:\Users\paint\AppData\Roaming\Python\Python39\site-packages\ortools\.libs\ortools.dll...


거리 계산 callback 함수 정의 및 등록 

In [2]:

#두 점 사이의 거리를 반환한다
def distance_callback(from_index,to_index):
    #내부 routing 인덱스를 실제 노드 번호로 변환 
    from_node=manager.IndexToNode(from_index)
    to_node=manager.IndexToNode(to_index)
    #거리 행렬에서 두 노드 간 거리 반환 
    return data['distance_matrix'][from_node][to_node]

#거리 계산 함수를 callback 함수로 등록하고 활용함
transit_callback_index=routing.RegisterTransitCallback(distance_callback) #함수를 솔버 내부에서 사용할 수 있는 형태로 등록 
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index) #솔버에게 경로(Arc)의 비용이 거리 기반으로 계산되게 설정함 

해 찾기   
[다른 전략](https://developers.google.com/optimization/routing/routing_options?hl=ko#first_solution_strategy)   
| 전략 이름               | 설명                          |
| ------------------- | --------------------------- |
| `PATH_CHEAPEST_ARC` | 현재 위치에서 가장 가까운 노드로 이동       |
| `SAVINGS`           | Clarke & Wright 절약 알고리즘 기반  |
| `CHRISTOFIDES`      | Christofides 근사 해법 (대칭 TSP) |
| `ALL_UNPERFORMED`   | 아무 노드도 방문하지 않는 해            |
| `AUTOMATIC`         | OR-Tools가 가장 적절한 전략을 자동 선택  |


In [3]:
#시작해를 찾기 위한 휴리스틱 메소드를 등록
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy=(
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    #시작점으로부터 가까운 노드를 추가하는 방식 
)
#최적해 출력 위한 함수 
def print_solution(manager,routing,solution):
    #총 거리 
    print(f'Objective : {solution.ObjectiveValue()}miles')
    #시작 노드 인덱스
    index= routing.Start(0)
    plan_output='Route for vehicle 0:\n'
    #총 경로 거리 누적값을 저장할 변수 
    route_distance=0
    #현재 인덱스가 vehicle의 경로 종로점인지 확인하는 반복문 
    while not routing.IsEnd(index):
        #현재 노드 번호를 경로 출력 문자열에 추가 
        plan_output+=f'{manager.IndexToNode(index)} ->'
        previous_index=index
        #다음 노드 인덱스를 솔루션에서 가져온다 
        index=solution.Value(routing.NextVar(index))
        #이전 노드부터 현재 노드까지 거리를 누적한다(previous_index,index,vehicle_id)
        route_distance += routing.GetArcCostForVehicle(previous_index,index,0)
    #마지막 노드를 경로 출력에 추가 
    plan_output+=f'{manager.IndexToNode(index)}\n'
    #전체 경로 정보를 출력 문자열에 추가 
    plan_output+=f'Route distance: {route_distance}miles\n'
    print(plan_output)
#문제풀이 실행 
solution= routing.SolveWithParameters(search_parameters) #설정한 파라미터로 최적 경로를 계산 
if solution: # 솔루션이 존재할 경우 결과 출력 
    print_solution(manager,routing,solution)#인덱스, 솔버, 솔루션 

Objective : 7293miles
Route for vehicle 0:
0 ->7 ->2 ->3 ->4 ->12 ->6 ->8 ->1 ->11 ->10 ->5 ->9 ->0
Route distance: 7293miles



해 경로를 리스트에 저장하기 

In [4]:
def get_routes(solution,routing,manager):
    routes=[]
    for route_nbr in range(routing.vehicles()):
        index=routing.Start(route_nbr)
        route=[manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index=solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes
routes=get_routes(solution,routing,manager)
for i, route in enumerate(routes):
    print('Route',i, route)

Route 0 [0, 7, 2, 3, 4, 12, 6, 8, 1, 11, 10, 5, 9, 0]


여러개의 차량을 사용할 수 있는 경우 

In [5]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model1():
    """Stores the data for the problem."""
    #data 딕셔너리 생성 
    data = {}

    #data['distance_matrix][i][j]: i에서 j까지 거리
    data["distance_matrix"] = [
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    ]

    data["num_vehicles"] = 4 #차량이 4대
    data["depot"] = 0
    return data
data=create_data_model1()
manager=pywrapcp.RoutingIndexManager(
    len(data['distance_matrix']),data['num_vehicles'],data['depot'])
routing= pywrapcp.RoutingModel(manager)

def distance_callback (from_index,to_index):
    from_node=manager.IndexToNode(from_index)
    to_node=manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]
transit_callback_index=routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

def print_solution(data, manager,routing,solution):
    print(f'Objective : {solution.ObjectiveValue()}miles')
    for i in range(data['num_vehicles']):
        route_distance=0
        plan_output=f'Route for vehicle {i}:\n'
        index=routing.Start(i)
        while not routing.IsEnd(index):
            plan_output+=f'{manager.IndexToNode(index)} -> '
            previous_index=index
            index=solution.Value(routing.NextVar(index))
            route_distance+=routing.GetArcCostForVehicle(previous_index,index,i)
        plan_output+=f'{manager.IndexToNode(index)}\n'
        plan_output+=f'Route distance: {route_distance}miles\n'
        print(plan_output)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy=(
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(data,manager,routing,solution)



Objective : 7569miles
Route for vehicle 0:
0 -> 0
Route distance: 0miles

Route for vehicle 1:
0 -> 0
Route distance: 0miles

Route for vehicle 2:
0 -> 0
Route distance: 0miles

Route for vehicle 3:
0 -> 7 -> 2 -> 3 -> 6 -> 8 -> 1 -> 11 -> 12 -> 4 -> 5 -> 10 -> 9 -> 0
Route distance: 7569miles



결과값을 보면 한 대의 차량을 이용하는 것이 가장 좋은 선택지이다.       

### 2. Capacity Constraint Vehicle Routing Problem (CVRP)
차량 용량 제약 

데이터 정의 

In [6]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        # fmt: off
      [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
      [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
      [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
      [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
      [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
      [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
      [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
      [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
      [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
      [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
      [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
      [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
      [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
      [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
      [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
      [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
      [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],
        # fmt: on
    ]
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data

경로 문제에서 시간이나 용량 문제를 다룰 때(CVRP, VRPTW) dimension 객체를 생성,설정하여 활용한다. 

**AddDimension** 메서드의 입력:

• **callback_index**  
&nbsp;&nbsp;&nbsp;&nbsp;노드 간 비용(거리, 시간, 수요 등)을 반환하는 콜백 함수의 인덱스,  
&nbsp;&nbsp;&nbsp;&nbsp;`RegisterTransitCallback` 메서드 등에서 생성하여 전달함

• **slack_max**  
&nbsp;&nbsp;&nbsp;&nbsp;여유를 나타내는 변수인 **slack**의 최댓값  
&nbsp;&nbsp;&nbsp;&nbsp;대기 시간이 필요 없는 문제에서는 일반적으로 `0`으로 설정

• **capacity**  
&nbsp;&nbsp;&nbsp;&nbsp;이 dimension의 누적값 상한  
&nbsp;&nbsp;&nbsp;&nbsp;예: 차량의 최대 적재량, 하루 최대 운행 시간 등 제한 조건

• **fix_start_cumulative_to_zero**  
&nbsp;&nbsp;&nbsp;&nbsp;`True`인 경우 누적값이 0에서 시작됨  
&nbsp;&nbsp;&nbsp;&nbsp;보통 `True`로 설정하며, 특수한 경우에만 `False` 사용

• **name**  
&nbsp;&nbsp;&nbsp;&nbsp;이 dimension의 이름 (예: `'Time'`, `'Capacity'`)  
&nbsp;&nbsp;&nbsp;&nbsp;프로그램의 다른 위치에서 참조할 때 사용


**slack**:
| 항목                | **CVRP (용량 제약)**                                 | **VRPTW (시간 창 제약)**                       |
| ----------------- | ------------------------------------------------ | ----------------------------------------- |
| **핵심 제약**         | 각 차량은 정해진 \*\*적재 용량(capacity)\*\*을 넘지 않아야 함      | 각 고객은 지정된 **시간 창(time window)** 안에 방문해야 함 |
| **Dimension의 의미** | 누적된 **적재량**                                      | 누적된 **도착 시간 (시간 경과)**                     |
| **Slack의 의미**     | 차량에 남은 **적재 여유량**                                | 고객에게 너무 **일찍 도착했을 때 대기 시간**               |
| **Slack이 필요한가?**  | ❌ (불필요) → **고객 수요만큼 정확히 줄면 됨** → `slack_max = 0` | ⭕                                         |
 

In [7]:
dimension_name='Distance'
routing.AddDimension(
    transit_callback_index,#callback_index
    0,#slack max
    3000,#capacity
    True,#fix_start_cumul_to_zero
    dimension_name
)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
#없는 dimension이름이면 즉시 에러를 발생시켜 프로그램이 죽게(die) 하겠다는 의도
distance_dimension.SetGlobalSpanCostCoefficient(100)
#Global span: 가장 멀리 간 차량의 누적 거리리-가장 적게 간 차량의 누적 거리리
#가장 긴 경로(거리/시간)에 큰 계수를 설정하면 목표 함수의 지배적인 요소가 되므로 프로그램은 가장 긴 경로의 길이를 최소화한다.
#최적해가 한쪽 차량만 너무 오래 달리는 걸 방지한다(부하 균형 효과)

출력함수 

In [8]:
def print_solution(data,manager,routing, solution):
    print(f'objective: {solution.ObjectiveValue()}')
    max_route_Distance =0
    for vehicle_id in range(data['num_vehicles']):
        index=routing.Start(vehicle_id)
        plan_output=f'Route for vehicle {vehicle_id}:\n'
        route_distance=0
        while not routing.IsEnd(index):
            plan_output+=f'{manager.IndexToNode(index)}->'
            previous_index=index
            index=solution.Value(routing.NextVar(index))
            route_distance+=routing.GetArcCostForVehicle(
                previous_index,index,vehicle_id
            )
        plan_output+=f'{manager.IndexToNode(index)}\n'
        plan_output+=f'Distance of the route: {route_distance}m\n'
        print(plan_output)
        max_route_Distance=max(route_distance,max_route_Distance)
    print(f'Maximum of the route distances: {max_route_Distance}m')

메인함수

In [9]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("No solution found !")

if __name__=='__main__':
    main()

objective: 177500
Route for vehicle 0:
0->9->10->2->6->5->0
Distance of the route: 1712m

Route for vehicle 1:
0->16->14->8->0
Distance of the route: 1484m

Route for vehicle 2:
0->7->1->4->3->0
Distance of the route: 1552m

Route for vehicle 3:
0->13->15->11->12->0
Distance of the route: 1552m

Maximum of the route distances: 1712m


### 3. Vehicle Routing Problem with Time Windows (VRPTW)
방문 가능한 시간 범위(Time window) 제약 

In [10]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    data = {}
    # 각 노드(고객) 간의 이동 시간 (단위: 분)
    data["time_matrix"] = [
        [0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
        [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
        [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
        [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
        [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
        [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
        [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
        [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
        [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
        [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
        [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
        [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
        [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
        [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
        [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
        [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
        [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0],
    ]

    # 각 노드의 시간창 (도착 가능한 시간 범위)
    data["time_windows"] = [
        (0, 5),    # 0: depot
        (7, 12),   # 1
        (10, 15),  # 2
        (16, 18),  # 3
        (10, 13),  # 4
        (0, 5),    # 5
        (5, 10),   # 6
        (0, 4),    # 7
        (5, 10),   # 8
        (0, 3),    # 9
        (10, 16),  # 10
        (10, 15),  # 11
        (0, 5),    # 12
        (5, 10),   # 13
        (7, 8),    # 14
        (10, 15),  # 15
        (11, 15),  # 16
    ]

    data["num_vehicles"] = 4
    data["depot"] = 0
    return data

프린트함수 

In [11]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}min")

**depot은 여러 차량이 공유하므로 따로 처리**

- 고객 노드 time window 설정 코드에서 depot은 `location_idx == 0` 하나뿐이므로 제외하고 (`continue`),이후 각 차량에 대해 **개별적으로 depot time window를 지정**

```python

if location_idx == data["depot"]:
    continue

# 각 차량의 출발 지점에 depot 시간창 설정
for vehicle_id in range(data["num_vehicles"]):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(
        data["time_windows"][0][0],
        data["time_windows"][0][1]
    )


In [12]:
def main():
    """Solve the VRP with time windows."""

    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix"]), data["num_vehicles"], data["depot"]
    )
    routing = pywrapcp.RoutingModel(manager)

    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["time_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    time = "Time"
    routing.AddDimension(
        transit_callback_index,
        30,  #slack(대기시간) 허용 
        30,  #최대운행시간
        False,  # Don't force start cumul to zero.
        time,
    )
    time_dimension = routing.GetDimensionOrDie(time)

    for location_idx, time_window in enumerate(data["time_windows"]):
        if location_idx == data["depot"]:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

    depot_idx = data["depot"]
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data["time_windows"][depot_idx][0], data["time_windows"][depot_idx][1]
        )

    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

Objective: 71
Route for vehicle 0:
0 Time(0,0) -> 9 Time(2,3) -> 14 Time(7,8) -> 16 Time(11,11) -> 0 Time(18,18)
Time of the route: 18min

Route for vehicle 1:
0 Time(0,0) -> 7 Time(2,4) -> 1 Time(7,11) -> 4 Time(10,13) -> 3 Time(16,16) -> 0 Time(24,24)
Time of the route: 24min

Route for vehicle 2:
0 Time(0,0) -> 12 Time(4,4) -> 13 Time(6,6) -> 15 Time(11,11) -> 11 Time(14,14) -> 0 Time(20,20)
Time of the route: 20min

Route for vehicle 3:
0 Time(0,0) -> 5 Time(3,3) -> 8 Time(5,5) -> 6 Time(7,7) -> 2 Time(10,10) -> 10 Time(14,14) -> 0 Time(20,20)
Time of the route: 20min

Total time of all routes: 82min


OPEN tour   
시작 지점으로 돌아올 필요가 없는 경로(생산 경로)    
가상 경로 도입해 잘라버리면 최적 생산 순서    

In [13]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    M=1000000
    data={}
    data['time']=[[M,10,17,15,0],
          [20,M,19,18,0],
          [50,44,M,22,0],
          [45,40,50,M,0],
          [0,0,0,0,M]]
    data['num_vehicles']=1  
    data['color']=['white','yellow','black','red','fake']
    data['depot']=4
    return data

def main():
    data=create_data_model()
    manager=pywrapcp.RoutingIndexManager(
        len(data['time']),data['num_vehicles'],data['depot']
    )
    routing=pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index,to_index):
        from_node=manager.IndexToNode(from_index)
        to_node=manager.IndexToNode(to_index)
        return data['time'][from_node][to_node]
    transit_callback_index=routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters=pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy=(
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    solution=routing.SolveWithParameters(search_parameters)
    return print_solution(manager,routing,solution)

def print_solution(manager, routing, solution):
    data=create_data_model()
    print(f"Objective: {solution.ObjectiveValue()} miles")
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    while not routing.IsEnd(index):
        node_index=manager.IndexToNode(index)
        if data['color'][node_index]!=data['color'][-1]:
            plan_output += f" {data['color'][node_index]} ->"
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output=plan_output.rstrip('->')
    plan_output += f"\nRoute distance: {route_distance}miles\n"
    print(plan_output)

if __name__=='__main__':
    main()

Objective: 51 miles
Route for vehicle 0:
 white -> yellow -> black -> red 
Route distance: 51miles

